<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center> 

# ¿Cómo ajustar BERT para la tarea de multi-etiquetado de textos?

En este notebook, aprenderemos a ajustar el modelo BERT para la tarea de multi-etiquetado de textos. 

Este notebook también se podría utilizar para ajustar otros modelos basados en BERT tales como RoBERTa, DistilBERT, etc, para esta tarea. 

Todos estos transformers trabajan de la misma manera: añaden una capa lineal sobre el modelo base, que va a generar un tensor  con el tamaño (batch_size, num_labels), indicando las probabilidades (no normalizadas) que corresponden a cada una de las etiquetas para cada uno de los ejemplos del batch (lote). 


En primer lugar, debemos instalar las librerías de transformers y datasets de HuggingFace.

In [1]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.3 MB/s eta 0:00:00


## Dataset

Vamos a descargar un dataset para la tarea de multi-etiquetado de HuggingFace. En concreto, vamos a utilizar el dataset **sem_eval_2018_task_1** formado con tweets que han sido etiquetados con una o varias de las siguientes emociones: 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'.

El dataset es multilingüe (árabe, inglés y español). Para este notebook, únicamente cargaremos el subconjunto para inglés.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")
dataset

Podemos ver que el dataset contiene 3 splits, y que su tamaño es relativamente pequeño. Vamos a mostrar un ejemplo:

In [5]:
import random
index = random.randint(0,dataset['train'].num_rows)
dataset['train'][index]


{'ID': '2017-En-21884',
 'Tweet': 'Mary Berry and her reign of terror',
 'anger': False,
 'anticipation': False,
 'disgust': False,
 'fear': True,
 'joy': False,
 'love': False,
 'optimism': False,
 'pessimism': False,
 'sadness': False,
 'surprise': False,
 'trust': False}

Creamos dos diccionarios que contiene el conjunto de etiquetas: 

In [6]:
labels = [label for label in dataset['train'].features.keys() if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'love',
 'optimism',
 'pessimism',
 'sadness',
 'surprise',
 'trust']

## Tokenization

Cargamos el tokenizador de BERT, y vamos a definir una función que además de tokenizar los textos también prepare las etiquetas de los textos de entrada. 
Como es un problema de multi-etiquetado, las etiquetas que corresponden a los textos de un lote, van a representarse en una matriz con dimensión (batch_size, num_labels). En concreto, esta matriz debería ser un tensor de números reales, porque de otra forma la funcón `BCEWithLogitsLoss` no funcionará correctamente. 

Ver: (https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [7]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # toma un lote de textos
  text = examples["Tweet"]
  # los codifica
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # Creamos un diccionario que para cada texto y para cada label, contenga los valores de las labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # creamos una matriz con la dimensión (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # modificamos la matriz
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  # añadimos el campo labels 
  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [9]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
encoded_dataset

Map:   0%|          | 0/6838 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6838
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3259
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 886
    })
})

Veamos un ejemplo al azar:

In [11]:
index = random.randint(0,encoded_dataset['train'].num_rows)
example = encoded_dataset['train'][index]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


Recuperamos su texto:

In [12]:
tokenizer.decode(example['input_ids'])

"[CLS] @ kateracculia @ themathofyou it's true! i reverse - engineered the drink. it's a scientific method. i hear it's all the rage. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

Y ahora recuperamos sus etiquetas:

In [13]:
example['labels']

[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [14]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['anger', 'joy', 'optimism']

Necesitamos transformar el formato del dataset para que tengan el formato de  PyTorch tensors. De esta forma, estamos transformando nuestros dataset a format PyTorch.

In [15]:
encoded_dataset.set_format("torch")

## Modelo
Con la función **from_pretrained** estamos cargando el modelo BERT base (es decir, los pesos de bert-base-uncased) con un cabezal de clasificación inicializado aleatorio (capa lineal) en la parte superior. 
Durante el proceso de fine-tuning, los pesos de esta cabeza se van a ajustar, junto con la base preentrenada a partir del dataset de la tarea. 

Necesitamos especificar en `problem_type` el valor "multi_label_classification", para asegurarnos que la función de perdida que se utiliza es la apropiada para el problema, es decir, que utilizará [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). 

También necesitamos indicar el número de etiquetas en la capa de salida, y los dos diccionarios id2label, label2id para hacer el mapping entre los ids y sus etiquetas.

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Para entrenar el modelo usaremos la clase Trainer. Este debe ser inicializado con un conjunto de hiperparámetros. (https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). 



In [19]:
from transformers import TrainingArguments, Trainer
batch_size = 8
metric_name = "f1"
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1, # recomendaos al menos 5
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

También debemos implementar nuestra función que sea capaz de calcular las métricas de evaluación sobre el conjunto de validación en cada epoch:

In [20]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Vamos a verificar el tipo de los datasets

In [21]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [22]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
         2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
        14354,  1001,  4105,  1001,  4737,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [23]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6477, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.2730, -0.1509,  0.0292, -0.3722, -0.4075,  0.0740,  0.2793,  0.2498,
         -0.4725, -0.0129, -0.1186]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Entrenamos:

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

## Evaluación
Evaluamos primero sobre el conjunto de validación:



In [26]:
trainer.evaluate()

{'eval_loss': 0.31123706698417664,
 'eval_f1': 0.7082096933728982,
 'eval_roc_auc': 0.8015983591180259,
 'eval_accuracy': 0.28442437923250563,
 'eval_runtime': 6.8159,
 'eval_samples_per_second': 129.99,
 'eval_steps_per_second': 16.285,
 'epoch': 5.0}

Ahora evaluamos sobre el conjunto test:

## Usar el modelo para inferir

Vamos a usar el modelo sobre una nueva oración: 

In [29]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

Los logits que produce el modelo tienen la dimensión (batch_size, num_labels). Como ahora únicamente estamos enviando una oración, el  `batch_size` es igual a 1. Los logits son un tensor que contiene las puntuaciones (no normalizadas) para cada etiqueta.

In [30]:
logits = outputs.logits
logits.shape

torch.Size([1, 11])

Para convertirlos en etiquetas reales, primero aplicamos una función sigmoidea de forma independiente a cada puntuación. De esta forma, cada puntuación se convierte en un número entre 0 y 1, que puede interpretarse como una "probabilidad" de cuán seguro es el modelo de que un la clase dada pertenece al texto de entrada.

A continuación, usamos un umbral (normalmente, 0,5) para convertir cada probabilidad en 1 (lo que significa que predecimos la etiqueta para el ejemplo dado) o en 0 (lo que significa que no predecimos la etiqueta para el ejemplo dado) ).

In [31]:
# usamos sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1

predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['joy', 'optimism']
